# Korean MRC Baseline

## Dependency
다음과 같은 라이브러리를 사용한다.
- [Konlpy](https://konlpy.org/ko/latest/index.html): 파이썬 한국어 NLP 처리기
- [Mecab-korean](https://bitbucket.org/eunjeon/mecab-ko-dic/src): 한국어 형태소 분석기

In [1]:
! apt-get install -y openjdk-8-jdk python3-dev
! pip install konlpy "tweepy<4.0.0"
! /bin/bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk openjdk-8-jdk-headless
  openjdk-8-jre openjdk-8-jre-headless x11-utils
0 upgraded,

In [2]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 14.5 MB/s 
     |████████████████████████████████| 895 kB 72.4 MB/s 
     |████████████████████████████████| 596 kB 72.6 MB/s 
     |████████████████████████████████| 6.5 MB 65.5 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
drive_path = '/content/drive/MyDrive/k-digital-goorm-3-korean-mrc/'

## 데이터셋 구성
현재 JSON 데이터를 볼 수 있는 클래스를 하나 작성하자.

In [5]:
from typing import List, Tuple, Dict, Any
import json
import random

class KoMRC:
    def __init__(self, data):
        self._data = data

    # Json을 불러오는 메소드
    @classmethod
    def load(cls, file_path: str):
        with open(file_path, 'r', encoding='utf-8') as fd:
            data = json.load(fd)

        _dataset = []
        for document in data['data']:
            for p_id, paragraph in enumerate(document['paragraphs']):
                context = paragraph['context']
                for q_id, qa in enumerate(paragraph['qas']):
                    question = qa['question']
                    for answer in qa['answers']:
                        answer_text = answer['text']
                        answer_start = answer['answer_start']
                        _dataset.append({'context':context,
                                        'question':question,
                                        'answer_start':answer_start,
                                         'answer_text':answer_text})            
        return cls(_dataset)

    # 기존에 잘라진 Json을 불러오는 메소드
    @classmethod
    def pre_load(cls, file_path: str):
        with open(file_path, 'r', encoding='utf-8') as js:
            _dataset = json.load(js)
        return cls(_dataset)

    # 데이터 셋을 잘라내는 메소드
    def split(self, eval_ratio: float=.1, seed=42):
        random.seed(seed)
        random.shuffle(self._data)
        train_dataset = dataset[int(len(self._data) * eval_ratio):]
        eval_dataset = dataset[:int(len(self._data) * eval_ratio)]
        return train_dataset, eval_dataset
    
    def __getitem__(self, index):
        return self._data[index]

    def __len__(self) -> int:
        return len(self._data)

`load` 메소드를 이용해서 Json 데이터를 불러올 수 있다.

In [6]:
dataset = KoMRC.load(drive_path + 'train.json')
print("Number of Samples:", len(dataset))
print(dataset[0])

Number of Samples: 17663
{'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.', 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?', 'answer_start': 478, 'answer_text': '한 달가량'}


In [7]:
dataset = KoMRC.pre_load(drive_path + 'train_truncated.json')
print("Number of Samples:", len(dataset))
print(dataset[0])

Number of Samples: 17663
{'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.', 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?', 'answer_text': '한 달가량', 'answer_start': 478}


`split` 메소드를 이용하면 데이터 셋을 나눌 수 있다.

In [ ]:
train_dataset, dev_dataset = KoMRC.split(dataset)
print("Number of Train Samples:", len(train_dataset))
print("Number of Dev Samples:", len(dev_dataset))
print(dev_dataset[0])

Number of Train Samples: 15897
Number of Dev Samples: 1766
{'context': "일까지 부산항국제전시컨벤션센터(BPEX)에서 진행되는 ‘부산인디커넥트페스티벌 2019(Busan Indie Connect Festival 2019, 이하 BIC 페스티벌)'에 참여한다고 3일 밝혔다. BIC 페스티벌은 우수한 국내 인디 게임들을 발굴, 육성하고, 인디 게임의 질적 향상을 도모하며, 글로벌 진출과 교류를 활성화하여 인디 게임들의 우수성을 널리 알리기 위한 목적으로 진행되는 글로벌 인디 게임 행사다. 2015년 진행된 1회 BIC 페스티벌부터 매년 후원을 해 온 에픽게임즈는 올해 역시 후원사로 참여하는 것은 물론 현장 부스에서 참관객들을 만날 예정이다. 또한, 에픽게임즈의 부스에서는 ‘에픽 온라인 서비스’와 인디 게임 <타워 오브 아르카나(Tower of Arcana)>를 만나볼 수 있다. ‘에픽 온라인 서비스’는 친구목록, 음성채팅, 업적 등 크로스 플랫폼 온라인 서비스에 필요한 다양한 프레임워크를 무료로 제공하는 서비스다. ‘에픽 온라인 서비스’에는 에픽게임즈가 주요 7개 플랫폼에서 2억 5천만 명이 넘는 사용자를 기반으로 <포트나이트> 서비스를 제공하면서 축적한 모든 기술과 노하우가 녹아있다. <타워 오브 아르카나>는 국내 개발사 펜타비트가 개발한 덱 빌딩 로그라이트 게임으로, 덱 빌딩 요소와 실시간 전투가 결합돼 새로운 재미를 주는 PC용 게임이다. 한편 언리얼 엔진으로 개발된 인디 게임인 터틀크림의 <RP6>와 데지마(Dejima)가 개발한 <파이어걸(FireGirl)>이 BIC 페스티벌 경쟁 부문에 선정됐다. <RP6>는 PC와 모바일, 콘솔에서 플레이할 수 있는 간단하고 추상적인 던전 탐험 게임이다. 플레이어는 7개의 슬롯을 돌려 게임 속 주인공을 간접적으로 컨트롤할 수 있으며, 절차적으로 생성되는 ‘슬롯 던전’은 플레이어가 매번 다른 경험을 할 수 있게 한다. <파이어걸>은 갑자기 불이 되살아나는 환상의 세

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bespin-global/klue-bert-base-aihub-mrc')

Downloading:   0%|          | 0.00/540 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

데이터를 Max_length에 맞춰주는 함수, 시간이 엄청 오래걸림...

In [ ]:
def data_truncation(data: KoMRC, tokenizer_, max_length=509): # context와 question을 토큰화하면, special token이 3개 붙기때문에 max_length에서 3을 빼줌
    count_truncated_item = 0
    for item in data:  # dataset 하나의 요소를 뽑아옴
        context_length = len(tokenizer_(item['context'], item['question'],add_special_tokens=False)['input_ids'])
        if context_length > max_length:
            q_len = len(tokenizer_(item['question'],add_special_tokens=False)['input_ids'])           
            count_truncated_item += 1
            cnt_for = 0
            start = item['answer_start']
            end = start + len(item['answer_text'])
            while context_length > max_length: # 토큰 개수가 512이하가 될 때 까지 반복
                # 정답이 context 중앙 기준 앞쪽에 있는 경우
                if len(tokenizer(item['context'][:start], add_special_tokens=False)['input_ids']) + len(tokenizer(item['answer_text'], add_special_tokens=False)['input_ids']) < (max_length - q_len)//2:
                    item['context'] = item['context'][:-3] # 맨 뒤에 두글자를 지워줌(answer start 지점 변동 없음)
                # 정답이 context 중앙 기준 뒤쪽에 있는 경우
                elif len(tokenizer(item['context'][end:], add_special_tokens=False)['input_ids']) + len(tokenizer(item['answer_text'], add_special_tokens=False)['input_ids']) < (max_length - q_len)//2:
                    item['context'] = item['context'][2:] # 맨 앞에 두글자를 지워줌
                    start -= 2 # answer start 지점 두칸 당겨줌
                    cnt_for += 2
                else:  # 정답 위치에서 context 기준 앞뒤로 여유가 있는 경우
                    item['context'] = item['context'][2:-3] # 맨 앞, 맨 뒤 두글자씩 지워줌
                    start -= 2 # answer start 지점 두칸 당겨줌               
                    end -= 2
                    cnt_for += 2 
                context_length = len(tokenizer(item['context'], item['question'])['input_ids'])
            item['answer_start'] -= cnt_for
            if item['context'][item['answer_start'] : item['answer_start']+len(item['answer_text'])] != item['answer_text']:
                print('truncation 실패...')
                print(item['context'], item['question'], item['answer_text'], item['answer_start'])
                print(item['context'][item['answer_start'] : item['answer_start']+len(item['answer_text'])])
                raise ValueError
        if count_truncated_item % 100 == 0:
            print('{}개 데이터 길이 조정'.format(count_truncated_item))
    return data

In [ ]:
dataset = data_truncation(dataset, tokenizer)

0개 데이터 길이 조정
0개 데이터 길이 조정
100개 데이터 길이 조정


KeyboardInterrupt: ignored

In [ ]:
cnt = 0
for item in dataset:
    if len(tokenizer(item['context'], item['question'])['input_ids']) > 512:
        cnt += 1
print(cnt)

0


In [ ]:
flag = True
for item in dataset:
    if item['context'][item['answer_start']:item['answer_start']+len(item['answer_text'])] != item['answer_text']:
        print('truncation 실패...')
        print(item)
        flag = False
        break
if flag:
    print('truncation 성공!!')

truncation 성공!!


In [ ]:
with open(drive_path + 'train_truncated.json', 'w', encoding='utf-8') as js:
    json.dump(dataset._data, js, indent=2, ensure_ascii=False)

In [ ]:
dataset = KoMRC.pre_load(drive_path + 'train_truncated.json')

In [ ]:
dataset

In [ ]:
string = '####test#####아아아아아'
re.sub('[#]','', string)

'test아아아아아'

In [ ]:
import re

tok_test = tokenizer.tokenize(dataset[0]['context'])
for i in tok_test:
    i = re.sub('[#]','', i)
    print(i)

일까
지
부산항
국제
전
시
컨
벤션
센터
(
B
P
EX
)
에서
진행
되
는
‘
부산
인
디
커
넥
트
페
스티벌
2019
(
Bus
an
Ind
ie
Con
ne
ct
Fe
st
iv
al
2019
,
이하
BI
C
페스티벌
)
'
에
참여
한다
고
3
일
밝혔
다
.
BI
C
페스티벌
은
우수
한
국내
인디
게임
들
을
발굴
,
육성
하고
,
인디
게임
의
질
적
향상
을
도모
하
며
,
글로벌
진출
과
교류
를
활성
화
하여
인디
게임
들
의
우수
성
을
널리
알리
기
위한
목적
으로
진행
되
는
글로벌
인디
게임
행사
다
.
2015
년
진행
된
1
회
BI
C
페스티벌
부터
매년
후원
을
해
온
에
픽
게임
즈
는
올해
역시
후원
사
로
참여
하
는
것
은
물론
현장
부스
에서
참관
객
들
을
만날
예정
이다
.
또한
,
에
픽
게임
즈
의
부스
에서
는
‘
에
픽
온라인
서비스
’
와
인디
게임
<
타워
오브
아르
카
나
(
To
we
r
of
Ar
ca
n
a
)
>
를
만나
볼
수
있
다
.
‘
에
픽
온라인
서비스
’
는
친구
목
록
,
음성
채
팅
,
업적
등
크로스
플랫폼
온라인
서비스
에
필요
한
다양
한
프레임
워크
를
무료
로
제공
하
는
서비스
다
.
‘
에
픽
온라인
서비스
’
에는
에
픽
게임
즈
가
주요
7
개
플랫폼
에서
2
억
5
천
만
명
이
넘
는
사용자
를
기반
으로
<
포트
나이
트
>
서비스
를
제공
하면
서
축적
한
모든
기술
과
노하우
가
녹
아
있
다
.
<
타워
오브
아르
카
나
>
는
국내
개발
사
펜
타
비
트
가
개발
한
덱
빌딩
로그
라이트
게임
으로
,
덱
빌딩
요소
와
실시간
전투
가
결합
돼
새로운
재미
를
주
는
PC
용
게임
이다
.
한편
언
리
얼
엔진
으로
개발
된
인디
게임
인
터
틀
크림
의
<
RP
6
>
와
데
지마
(
De
j
im
a
)
가
개발
한
<
파이어
걸
(
F
ire
G
irl
)
>
이
BI
C
페스티벌
경쟁
부문
에
선정
됐
다
.
<
RP
6
>
는
PC
와


In [ ]:
dataset[0]

{'answer_start': 757,
 'answer_text': '<RP6>',
 'context': "일까지 부산항국제전시컨벤션센터(BPEX)에서 진행되는 ‘부산인디커넥트페스티벌 2019(Busan Indie Connect Festival 2019, 이하 BIC 페스티벌)'에 참여한다고 3일 밝혔다. BIC 페스티벌은 우수한 국내 인디 게임들을 발굴, 육성하고, 인디 게임의 질적 향상을 도모하며, 글로벌 진출과 교류를 활성화하여 인디 게임들의 우수성을 널리 알리기 위한 목적으로 진행되는 글로벌 인디 게임 행사다. 2015년 진행된 1회 BIC 페스티벌부터 매년 후원을 해 온 에픽게임즈는 올해 역시 후원사로 참여하는 것은 물론 현장 부스에서 참관객들을 만날 예정이다. 또한, 에픽게임즈의 부스에서는 ‘에픽 온라인 서비스’와 인디 게임 <타워 오브 아르카나(Tower of Arcana)>를 만나볼 수 있다. ‘에픽 온라인 서비스’는 친구목록, 음성채팅, 업적 등 크로스 플랫폼 온라인 서비스에 필요한 다양한 프레임워크를 무료로 제공하는 서비스다. ‘에픽 온라인 서비스’에는 에픽게임즈가 주요 7개 플랫폼에서 2억 5천만 명이 넘는 사용자를 기반으로 <포트나이트> 서비스를 제공하면서 축적한 모든 기술과 노하우가 녹아있다. <타워 오브 아르카나>는 국내 개발사 펜타비트가 개발한 덱 빌딩 로그라이트 게임으로, 덱 빌딩 요소와 실시간 전투가 결합돼 새로운 재미를 주는 PC용 게임이다. 한편 언리얼 엔진으로 개발된 인디 게임인 터틀크림의 <RP6>와 데지마(Dejima)가 개발한 <파이어걸(FireGirl)>이 BIC 페스티벌 경쟁 부문에 선정됐다. <RP6>는 PC와 모바일, 콘솔에서 플레이할 수 있는 간단하고 추상적인 던전 탐험 게임이다. 플레이어는 7개의 슬롯을 돌려 게임 속 주인공을 간접적으로 컨트롤할 수 있으며, 절차적으로 생성되는 ‘슬롯 던전’은 플레이어가 매번 다른 경험을 할 수 있게 한다. <파이어걸>은 갑자기 불이 되살아나는 환상의 세계를 배경으로 한 액션

In [ ]:
sentence = dataset[0]['context']
position = 0
tokens= []
for morph in tokenizer.tokenize(sentence):
    morph = re.sub('[#]','', morph)
    position = sentence.find(morph, position)
    tokens.append((morph, (position, position + len(morph))))
    position += len(morph)
print(tokens)

[('일까', (0, 2)), ('지', (2, 3)), ('부산항', (4, 7)), ('국제', (7, 9)), ('전', (9, 10)), ('시', (10, 11)), ('컨', (11, 12)), ('벤션', (12, 14)), ('센터', (14, 16)), ('(', (16, 17)), ('B', (17, 18)), ('P', (18, 19)), ('EX', (19, 21)), (')', (21, 22)), ('에서', (22, 24)), ('진행', (25, 27)), ('되', (27, 28)), ('는', (28, 29)), ('‘', (30, 31)), ('부산', (31, 33)), ('인', (33, 34)), ('디', (34, 35)), ('커', (35, 36)), ('넥', (36, 37)), ('트', (37, 38)), ('페', (38, 39)), ('스티벌', (39, 42)), ('2019', (43, 47)), ('(', (47, 48)), ('Bus', (48, 51)), ('an', (51, 53)), ('Ind', (54, 57)), ('ie', (57, 59)), ('Con', (60, 63)), ('ne', (63, 65)), ('ct', (65, 67)), ('Fe', (68, 70)), ('st', (70, 72)), ('iv', (72, 74)), ('al', (74, 76)), ('2019', (77, 81)), (',', (81, 82)), ('이하', (83, 85)), ('BI', (86, 88)), ('C', (88, 89)), ('페스티벌', (90, 94)), (')', (94, 95)), ("'", (95, 96)), ('에', (96, 97)), ('참여', (98, 100)), ('한다', (100, 102)), ('고', (102, 103)), ('3', (104, 105)), ('일', (105, 106)), ('밝혔', (107, 109)), ('다', (109, 110)), ('.

단어 단위로 토큰화해서 정답 위치를 찾기 위하여 토큰화 및 위치 인덱싱을 하는 클래스를 상속을 통해 작성해 보자.

In [ ]:
context, position = zip(*_tokenize_with_position(dataset[0]['context'], tokenizer))

In [ ]:
context, position

(('일까',
  '지',
  '부산항',
  '국제',
  '전',
  '시',
  '컨',
  '벤션',
  '센터',
  '(',
  'B',
  'P',
  'EX',
  ')',
  '에서',
  '진행',
  '되',
  '는',
  '‘',
  '부산',
  '인',
  '디',
  '커',
  '넥',
  '트',
  '페',
  '스티벌',
  '2019',
  '(',
  'Bus',
  'an',
  'Ind',
  'ie',
  'Con',
  'ne',
  'ct',
  'Fe',
  'st',
  'iv',
  'al',
  '2019',
  ',',
  '이하',
  'BI',
  'C',
  '페스티벌',
  ')',
  "'",
  '에',
  '참여',
  '한다',
  '고',
  '3',
  '일',
  '밝혔',
  '다',
  '.',
  'BI',
  'C',
  '페스티벌',
  '은',
  '우수',
  '한',
  '국내',
  '인디',
  '게임',
  '들',
  '을',
  '발굴',
  ',',
  '육성',
  '하고',
  ',',
  '인디',
  '게임',
  '의',
  '질',
  '적',
  '향상',
  '을',
  '도모',
  '하',
  '며',
  ',',
  '글로벌',
  '진출',
  '과',
  '교류',
  '를',
  '활성',
  '화',
  '하여',
  '인디',
  '게임',
  '들',
  '의',
  '우수',
  '성',
  '을',
  '널리',
  '알리',
  '기',
  '위한',
  '목적',
  '으로',
  '진행',
  '되',
  '는',
  '글로벌',
  '인디',
  '게임',
  '행사',
  '다',
  '.',
  '2015',
  '년',
  '진행',
  '된',
  '1',
  '회',
  'BI',
  'C',
  '페스티벌',
  '부터',
  '매년',
  '후원',
  '을',
  '해',
  '온',
  '에',
  '픽

In [ ]:
data = dataset[0]
data['context']

"일까지 부산항국제전시컨벤션센터(BPEX)에서 진행되는 ‘부산인디커넥트페스티벌 2019(Busan Indie Connect Festival 2019, 이하 BIC 페스티벌)'에 참여한다고 3일 밝혔다. BIC 페스티벌은 우수한 국내 인디 게임들을 발굴, 육성하고, 인디 게임의 질적 향상을 도모하며, 글로벌 진출과 교류를 활성화하여 인디 게임들의 우수성을 널리 알리기 위한 목적으로 진행되는 글로벌 인디 게임 행사다. 2015년 진행된 1회 BIC 페스티벌부터 매년 후원을 해 온 에픽게임즈는 올해 역시 후원사로 참여하는 것은 물론 현장 부스에서 참관객들을 만날 예정이다. 또한, 에픽게임즈의 부스에서는 ‘에픽 온라인 서비스’와 인디 게임 <타워 오브 아르카나(Tower of Arcana)>를 만나볼 수 있다. ‘에픽 온라인 서비스’는 친구목록, 음성채팅, 업적 등 크로스 플랫폼 온라인 서비스에 필요한 다양한 프레임워크를 무료로 제공하는 서비스다. ‘에픽 온라인 서비스’에는 에픽게임즈가 주요 7개 플랫폼에서 2억 5천만 명이 넘는 사용자를 기반으로 <포트나이트> 서비스를 제공하면서 축적한 모든 기술과 노하우가 녹아있다. <타워 오브 아르카나>는 국내 개발사 펜타비트가 개발한 덱 빌딩 로그라이트 게임으로, 덱 빌딩 요소와 실시간 전투가 결합돼 새로운 재미를 주는 PC용 게임이다. 한편 언리얼 엔진으로 개발된 인디 게임인 터틀크림의 <RP6>와 데지마(Dejima)가 개발한 <파이어걸(FireGirl)>이 BIC 페스티벌 경쟁 부문에 선정됐다. <RP6>는 PC와 모바일, 콘솔에서 플레이할 수 있는 간단하고 추상적인 던전 탐험 게임이다. 플레이어는 7개의 슬롯을 돌려 게임 속 주인공을 간접적으로 컨트롤할 수 있으며, 절차적으로 생성되는 ‘슬롯 던전’은 플레이어가 매번 다른 경험을 할 수 있게 한다. <파이어걸>은 갑자기 불이 되살아나는 환상의 세계를 배경으로 한 액션 플랫폼 PC 게임으로, 플레이어는 파이어걸과 함께 도시를 탐험하면서 몬스터의 비밀을 풀어야 한다."

In [ ]:
from typing import Generator

import konlpy
import re
class TokenizedKoMRC(KoMRC):
    def __init__(self, data) -> None:
        super().__init__(data)
        self._tokenizer = AutoTokenizer.from_pretrained('bespin-global/klue-bert-base-aihub-mrc')

    def _tokenize_with_position(self, data):
        position = 0
        tokens = []
        sentence = data['context']
        for morph in self._tokenizer.tokenize(sentence):
            morph = re.sub('[#]','', morph)
            position = sentence.find(morph, position)
            tokens.append((morph, (position, position + len(morph))))
            position += len(morph)
        return tokens

    def set_tokenized_position(self):
        for index, data in enumerate(self._data):
            context, position = zip(*self._tokenize_with_position(data))
            context, position = list(context), list(position)
            answer = data['answer_text']
            for idx, (position_start, position_end) in enumerate(position):
                if position_start <= data['answer_start'] < position_end:
                    start = idx
                    break
            else:
                print(context)
                print(answer)
                raise ValueError("No mathced start position")
            target = ''.join(answer.split(' '))
            source = ''
            for n, morph in enumerate(context[start:]):
                if morph == '[UNK]':
                    morph = target[len(source)]
                source += morph
                if target in source:
                    break
            else:
                print(target)
                print(source)
                raise ValueError("No Matched end position")

            answers = {'start': start,
                        'end': start+n}
            self._data[index]['answer_positions'] = answers

In [ ]:
dataset = KoMRC.pre_load(drive_path + 'train_truncated.json')
dataset = TokenizedKoMRC(dataset)

In [ ]:
dataset.set_tokenized_position()

['화질', '(', 'UHD', ')', '곡', '면', 'TV', '를', '출시', '했', '다', '.', '디스플레이', '도', '한국', '이나', '대만', '산', '이', '아닌', '중국', '‘', '차이나', '스타', '’', '제품', '을', '채용', '했', '다', '.', '순수', '중국', '기술', '이', '란', '점', '을', '과시', '한', '것', '이다', '.', 'TC', 'L', '과', '중국', '의', '하이', '센스', '는', '[UNK]', '(', '양자', '점', ')', 'TV', '와', 'OLED', '(', '유기', '발', '광', '다이', '오드', ')', 'TV', '도', '동시', '에', '전시', '했', '다', '.', 'TC', 'L', '관계자', '는', '“', '둘', '다', '기존', 'LCD', '(', '액정', '표', '시장', '치', ')', 'TV', '보', '다', '두께', '가', '얇', '다', '”', '며', '“', 'OLED', '는', '고급', '형', ',', '[UNK]', '중', '가', '형', '으로', '내년', '중', '정식', '출시', '할', '것', '”', '이라', '고', '말', '했', '다', '.', '이', '관계자', '는', '‘', 'TC', 'L', '이', '기술', '을', '선도', '하고', '있', '다고', '생각', '하', '느', '냐', '’', '는', '기자', '의', '질문', '에', '“', '그렇', '다', '”', '고', '자신', '했', '다', '.', '화질', '등', '전반', '적인', '품질', '은', '한국', '산', '에', '못', '미치', '지만', '더', '이상', '후발', '주', '자', '로', '머물', '지', '않', '겠다', '는', '의지', '를', '분명히', '한', '것

ValueError: ignored

In [ ]:
tokenized_mrc = TokenizedKoMRC(dataset, tokenizer)
dataset = TokenizedKoMRC.load(drive_path + 'train.json')

#train_dataset, dev_dataset = TokenizedKoMRC.split(dataset)
#print("Number of Train Samples:", len(train_dataset))
#print("Number of Dev Samples:", len(dev_dataset))
#print(dev_dataset[0])

In [ ]:
sample = dev_dataset[0]
print(sample['context'][sample['answers'][0]['start']:sample['answers'][0]['end']+1])
print(len (sample['context']))

['서강대', '메리', '홀', '대극장']
695


## Vocab 생성 및 Indexing
토큰화된 데이터 셋을 기준으로 Vocab을 만들고 인덱싱을 하는 `Indexer`를 만들자.

In [ ]:
from typing import Sequence
from collections import Counter
from itertools import chain

from tqdm.notebook import tqdm

class Indexer:
    def __init__(self,
        id2token: List[str], 
        max_length: int=1024,
        pad: str='<pad>', unk: str='<unk>', cls: str='<cls>', sep: str='<sep>'
    ):
        self.pad = pad
        self.unk = unk
        self.cls = cls
        self.sep = sep
        self.special_tokens = [pad, unk, cls, sep]

        self.max_length = max_length

        self.id2token = self.special_tokens + id2token
        self.token2id = {token: token_id for token_id, token in enumerate(self.id2token)}

    @property
    def vocab_size(self):
        return len(self.id2token)
    
    @property
    def pad_id(self):
        return self.token2id[self.pad]
    @property
    def unk_id(self):
        return self.token2id[self.unk]
    @property
    def cls_id(self):
        return self.token2id[self.cls]
    @property
    def sep_id(self):
        return self.token2id[self.sep]

    @classmethod
    def build_vocab(cls,
        dataset: TokenizedKoMRC, 
        min_freq: int=5
    ):
        counter = Counter(chain.from_iterable(
            sample['context'] + sample['question']
            for sample in tqdm(dataset, desc="Counting Vocab")
        ))

        return cls([word for word, count in counter.items() if count >= min_freq])
    
    def decode(self,
        token_ids: Sequence[int]
    ):
        return [self.id2token[token_id] for token_id in token_ids]

    def sample2ids(self,
        sample: Dict[str, Any],
    ) -> Dict[str, Any]:
        context = [self.token2id.get(token, self.unk_id) for token in sample['context']]
        question = [self.token2id.get(token, self.unk_id) for token in sample['question']]

        context = context[:self.max_length-len(question)-3]             # Truncate context
        
        input_ids = [self.cls_id] + question + [self.sep_id] + context + [self.sep_id]
        token_type_ids = [0] * (len(question) + 1) + [1] * (len(context) + 2)

        if sample['answers'] is not None:
            answer = sample['answers'][0]
            start = min(answer['start'] + len(question) + 2, self.max_length - 1)
            end = min(answer['end'] + len(question) + 2, self.max_length - 1)
        else:
            start = None
            end = None

        return {
            'guid': sample['guid'],
            'context': sample['context_original'],
            'question': sample['question_original'],
            'position': sample['context_position'],
            'input_ids': input_ids,
            'token_type_ids': token_type_ids,
            'start': start,
            'end': end
        }

In [ ]:
indexer = Indexer.build_vocab(dataset)
print(indexer.sample2ids(dev_dataset[0]))

Counting Vocab:   0%|          | 0/12037 [00:00<?, ?it/s]

{'guid': '844e22ab28924c1697d5ac28801b34c1', 'context': '지난해 주요 연극상을 나눠 가졌던 세 편의 작품이 올봄에 나란히 앙코르 무대를 갖는다. 대한민국연극대상 연기·무대예술상, 동아연극상 작품·희곡·연기상 등을 수상한 ‘알리바이 연대기’(17~20일 대학로 아르코예술극장 대극장, 25일~5월11일 서계동 국립극단 백성희장민호극장), 연극대상에서 대상과 희곡상을 받은 ‘여기가 집이다’(18일~5월22일 대학로 연우소극장), 연극대상 작품·연출상과 김상열연극상 수상작인 ‘황금용’(5월9~18일 서강대 메리홀 대극장)이다. 초연 당시 짧은 상연 기간과 낮은 인지도 등으로 공연을 놓친 연극팬에겐 평단으로부터 작품성을 인정받은 수작을 관람할 수 있는 기회다. ‘알리바이 연대기’는 희곡을 쓰고 연출한 김재엽의 가족사에 근거한 다큐멘터리 드라마다. 1930년에 태어난 한 개인의 사적인 연대기를 바탕으로 그 사이를 파고드는 역사적 순간들을 정밀하게 조명한다. 연출가는 “공적인 권력이 사적인 권리를 지켜주기보다 억압하기 일쑤였던 한국 현대사 속에서 개인은 언제나 무죄를 입증하며 하루하루 자신을 지켜내야 하는 ‘알리바이의 연대기’ 속에서 살아왔다”고 말한다.한국연극평론가협회는 이 작품을 ‘2013년 올해의 연극 베스트3’로 선정하며 “촘촘하고 세세하게 삶에 천착해 개인과 역사에 대한 이분법적 관점을 극복한다. 정치극에 대한 새로운 가능성을 보여줬다”고 평했다. 이 작품으로 연기상을 휩쓴 남명렬을 비롯해 지춘성 정원조 등 초연 배우들이 그대로 출연한다.‘여기가 집이다’는 허름하고 볼품 없는 ‘20년 전통’의 고시원에 모여 사는 사람들의 절망과 희망을 그린 작품. ‘차력사와 아코디언’ ‘택배 왔어요’를 만든 극단 이와삼의 장우재 대표가 직접 대본을 쓰고 연출했다. 나름의 규칙을 가지고 평화로웠던 고시원에 새로운 주인으로 등장한 ‘20세 고등학생’ 동교가 “이제부터 고시원 식구들에게 월세를 받지 않겠다”고 선언하면서 갑작스런 변화의 바람

쉽게 Indexer를 활용하기 위해 Indexer가 포함된 데이터 셋을 만들자.

In [ ]:
class IndexerWrappedDataset:
    def __init__(self, dataset: TokenizedKoMRC, indexer: Indexer) -> None:
        self._dataset = dataset
        self._indexer = indexer

    def __len__(self) -> int:
        return len(self._dataset)
    
    def __getitem__(self, index: int) -> Dict[str, Any]:
        sample = self._indexer.sample2ids(self._dataset[index])
        sample['attention_mask'] = [1] * len(sample['input_ids'])

        return sample


In [ ]:
indexed_train_dataset = IndexerWrappedDataset(train_dataset, indexer)
indexed_dev_dataset = IndexerWrappedDataset(dev_dataset, indexer)

sample = indexed_dev_dataset[0]
print(sample['input_ids'], sample['attention_mask'], sample['token_type_ids'], sample['start'], sample['end'])
print(len(sample['input_ids']), len(sample['attention_mask']))

[2, 1, 1100, 7, 2226, 71, 4336, 30, 25656, 1139, 97, 57, 39, 2874, 39, 2684, 974, 170, 3, 67, 2912, 4336, 830, 97, 1250, 1278, 553, 1607, 2344, 41, 2239, 30, 4, 3276, 32, 6946, 25656, 4344, 82, 2339, 1386, 15, 1016, 4336, 257, 4355, 695, 4344, 4335, 830, 66, 2412, 4336, 830, 2239, 695, 2386, 695, 4355, 830, 17, 97, 834, 71, 183, 36128, 12542, 588, 8, 64, 90, 9, 276, 142, 1, 4335, 4333, 29910, 66, 132, 9, 64, 246, 130, 530, 9, 24315, 19208, 1, 29106, 4333, 175, 66, 4336, 257, 11, 257, 74, 6503, 1, 97, 649, 20, 183, 2830, 7, 635, 30, 14, 588, 85, 9, 64, 246, 130, 1375, 9, 276, 142, 1, 19205, 175, 66, 4336, 257, 2239, 695, 2226, 830, 74, 33837, 4336, 830, 834, 28079, 183, 1, 588, 246, 130, 1513, 64, 85, 9, 6741, 1807, 1011, 29910, 175, 30, 14, 15, 9493, 1293, 2500, 20, 5450, 135, 74, 981, 20, 2426, 17, 43, 1139, 97, 11653, 4336, 2839, 7081, 9478, 2462, 2239, 332, 97, 4133, 20, 17011, 97, 12211, 407, 402, 38, 39, 950, 14, 15, 183, 36128, 12542, 191, 39, 2386, 97, 2286, 243, 2226, 71, 1, 41

In [ ]:
indexed_train_dataset = tokenizer(train_dataset, max_length=512, truncation=True, padding='max_length', return_token_type_ids=False, return_offsets_mapping=True)
indexed_dev_dataset = tokenizer(train_dataset, max_length=512, truncation=True, padding='max_length', return_token_type_ids=False, return_offsets_mapping=True)

In [ ]:
max_length = 0
for i in indexed_train_dataset:
    if len(i['input_ids']) > max_length:
        max_length = len(i['input_ids'])
        if len(i['input_ids'])==1024:
            print(i['input_ids'])
print(max_length)

[2, 584, 130, 1007, 9, 16186, 3995, 97, 17356, 3445, 39, 170, 3, 1877, 787, 1277, 520, 19666, 7, 3495, 82, 2117, 475, 4412, 41, 248, 48, 13791, 1352, 20, 145, 41, 4728, 32, 19128, 5688, 82, 10919, 57, 243, 38, 133, 14, 15, 13638, 264, 20, 22528, 71, 3767, 482, 14, 15, 4131, 3266, 57, 243, 520, 19666, 39, 2830, 920, 22534, 475, 142, 2292, 242, 14, 15, 584, 130, 1007, 9, 66, 520, 19666, 39, 15957, 2419, 32, 16186, 3995, 97, 501, 242, 14, 15, 18078, 20, 6261, 82, 3578, 2077, 7095, 66, 10671, 30, 2434, 8733, 1352, 97, 16186, 71, 99, 30, 14, 15, 13791, 1352, 20, 13652, 74, 1, 151, 1, 142, 22535, 3995, 71, 1815, 6344, 1179, 43, 4051, 242, 14, 15, 22536, 71, 8733, 1352, 20, 1315, 97, 4788, 258, 97, 22537, 1521, 11672, 32, 39, 3183, 86, 167, 133, 14, 15, 11692, 6100, 13791, 1352, 20, 20836, 22538, 59, 41, 6868, 142, 692, 2611, 116, 7828, 242, 14, 15, 4212, 557, 8733, 1352, 20, 467, 43, 5733, 17032, 41, 571, 97, 1277, 116, 80, 133, 14, 15, 30, 17032, 97, 2552, 13791, 1352, 20, 3305, 8722, 145, 

## Transformer Encoder를 활용한 MRC 모델
![Bert for MRC](https://miro.medium.com/max/340/1*cXDOP0gsE7Zp8-sgZqYfTA.png)

Transformer 인코더 마지막에 Linear Layer를 붙여 정답의 시작과 끝을 맞추는 간단한 모델을 생성보자.

In [ ]:
import torch.nn as nn

from transformers.models.bert.modeling_bert import (
    BertModel,
    BertPreTrainedModel,
    Bert
)

## Simple Version for Bert QA: https://huggingface.co/transformers/_modules/transformers/models/bert/modeling_bert.html#BertForQuestionAnswering.forward
class BertForQuestionAnswering(BertPreTrainedModel):
    _keys_to_ignore_on_load_unexpected = [r"pooler"]

    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config, add_pooling_layer=False)
        self.start_linear = nn.Linear(config.hidden_size, 1)
        self.end_linear = nn.Linear(config.hidden_size, 1)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None
    ):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        start_logits = self.start_linear(outputs.last_hidden_state).squeeze(-1)
        end_logits = self.end_linear(outputs.last_hidden_state).squeeze(-1)

        return start_logits, end_logits

## 학습 준비

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

class Collator:
    def __init__(self, indexer: Indexer) -> None:
        self._indexer = indexer

    def __call__(self, samples: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        samples = {
            key: [sample[key] for sample in samples]
            for key in samples[0]
        }

        for key in 'start', 'end':
            if samples[key][0] is None:
                samples[key] = None
            else:
                samples[key] = torch.tensor(samples[key], dtype=torch.long)
        for key in 'input_ids', 'attention_mask', 'token_type_ids':
            samples[key] = pad_sequence(
                [torch.tensor(sample, dtype=torch.long) for sample in samples[key]],
                batch_first=True, padding_value=self._indexer.pad_id
            )

        return samples

In [ ]:
from torch.utils.data import DataLoader

batch_size = 64
accumulation = 4 # 메모리를 아끼기 위하여 Gradient accumulation을 해보자

collator = Collator(indexer)
train_loader = DataLoader(indexed_train_dataset, batch_size=batch_size//accumulation, shuffle=True, collate_fn=collator, num_workers=2)
dev_loader = DataLoader(indexed_dev_dataset, batch_size=batch_size//accumulation, shuffle=False, collate_fn=collator, num_workers=2)

In [ ]:
batch = next(iter(dev_loader))
print(batch['input_ids'].shape)
print(batch['input_ids'])
print(list(batch.keys()))

In [ ]:
import torch
from transformers import BertConfig

torch.manual_seed(42)
config = BertConfig(
     vocab_size=indexer.vocab_size,
     max_position_embeddings=1024,
     hidden_size=256,
     num_hidden_layers=4,
     num_attention_heads=4,
     intermediate_size=1024
)
model = BertForQuestionAnswering(config)
model.cuda()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [ ]:
import os
from statistics import mean

import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_

os.makedirs('dump', exist_ok=True)
train_losses = []
dev_losses = []

step = 0

for epoch in range(1, 31):
    print("Epoch", epoch)
    # Training
    running_loss = 0.
    losses = []
    progress_bar = tqdm(train_loader, desc='Train')
    for batch in progress_bar:
        del batch['guid'], batch['context'], batch['question'], batch['position']
        # batch = {key: value.cuda() for key, value in batch.items()}
        start = batch.pop('start')
        end = batch.pop('end')
        
        start_logits, end_logits = model(**batch)
        loss = F.cross_entropy(start_logits, start) + F.cross_entropy(end_logits, end)
        (loss / accumulation).backward()
        running_loss += loss.item()
        del batch, start, end, start_logits, end_logits, loss
        
        step += 1
        if step % accumulation:
            continue

        clip_grad_norm_(model.parameters(), max_norm=1.)
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)

        losses.append(running_loss / accumulation)
        running_loss = 0.
        progress_bar.set_description(f"Train - Loss: {losses[-1]:.3f}")
    train_losses.append(mean(losses))
    print(f"train score: {train_losses[-1]:.3f}")

    # Evaluation
    losses = []
    for batch in tqdm(dev_loader, desc="Evaluation"):
        del batch['guid'], batch['context'], batch['question'], batch['position']
        # batch = {key: value.cuda() for key, value in batch.items()}
        start = batch.pop('start')
        end = batch.pop('end')
        
        with torch.no_grad():
            start_logits, end_logits = model(**batch)
        loss = F.cross_entropy(start_logits, start) + F.cross_entropy(end_logits, end)

        losses.append(loss.item())
        del batch, start, end, start_logits, end_logits, loss
    dev_losses.append(mean(losses))
    print(f"Evaluation score: {dev_losses[-1]:.3f}")

    model.save_pretrained(f'dump/model.{epoch}')

In [ ]:
import matplotlib.pyplot as plt

t = list(range(1, 31))
plt.plot(t, train_losses, label="Train Loss")
plt.plot(t, dev_losses, label="Dev Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()


학습 데이터 셋에 Overfitting이 일어나는 것을 확인할 수 있다.

## Answer Inference
모델의 Output을 활용해서 질문의 답을 찾는 코드를 작성하자.

In [ ]:
model = BertForQuestionAnswering.from_pretrained('dump/model.30')
model.cuda()
model.eval()

In [ ]:
for idx, sample in zip(range(1, 4), indexed_train_dataset):
    print(f'------{idx}------')
    print('Context:', sample['context'])
    print('Question:', sample['question'])
    
    input_ids, token_type_ids = [
        torch.tensor(sample[key], dtype=torch.long, device="cuda")
        for key in ("input_ids", "token_type_ids")
    ]
    
    with torch.no_grad():
        start_logits, end_logits = model(input_ids=input_ids[None, :], token_type_ids=token_type_ids[None, :])
    start_logits.squeeze_(0), end_logits.squeeze_(0)
    
    start_prob = start_logits[token_type_ids.bool()][1:-1].softmax(-1)
    end_prob = end_logits[token_type_ids.bool()][1:-1].softmax(-1)
    probability = torch.triu(start_prob[:, None] @ end_prob[None, :])
    index = torch.argmax(probability).item()
    
    start = index // len(end_prob)
    end = index % len(end_prob)
    
    start = sample['position'][start][0]
    end = sample['position'][end][1]

    print('Answer:', sample['context'][start:end])

## Test 출력 파일 작성

In [ ]:
test_dataset = TokenizedKoMRC.load('/kaggle/input/k-digital-goorm-3-korean-mrc/test.json')
test_dataset = IndexerWrappedDataset(test_dataset, indexer)
print("Number of Test Samples", len(test_dataset))
print(test_dataset[0])

In [ ]:
import csv

os.makedirs('out', exist_ok=True)
with torch.no_grad(), open('out/baseline.csv', 'w') as fd:
    writer = csv.writer(fd)
    writer.writerow(['Id', 'Predicted'])

    rows = []
    for sample in tqdm(test_dataset, "Testing"):
        input_ids, token_type_ids = [
            torch.tensor(sample[key], dtype=torch.long, device="cuda")
            for key in ("input_ids", "token_type_ids")
        ]
    
        with torch.no_grad():
            start_logits, end_logits = model(input_ids=input_ids[None, :], token_type_ids=token_type_ids[None, :])
        start_logits.squeeze_(0), end_logits.squeeze_(0)
    
        start_prob = start_logits[token_type_ids.bool()][1:-1].softmax(-1)
        end_prob = end_logits[token_type_ids.bool()][1:-1].softmax(-1)
        probability = torch.triu(start_prob[:, None] @ end_prob[None, :])
        index = torch.argmax(probability).item()
    
        start = index // len(end_prob)
        end = index % len(end_prob)
    
        start = sample['position'][start][0]
        end = sample['position'][end][1]

        rows.append([sample["guid"], sample['context'][start:end]])
    
    writer.writerows(rows)